# Preprocess data

## Load data

In [8]:
import gzip
interactions = {}
data = []
# Load data
org_id = '4932' # Change to 9606 for Human

with gzip.open(f'data/{org_id}.protein.links.v11.0.txt.gz', 'rt') as f:
    next(f) # Skip header
    for line in f:
        p1, p2, score = line.strip().split()
        if float(score) < 700: # Filter high confidence interactions
            continue
        if p1 not in interactions:
            interactions[p1] = set()
        if p2 not in interactions:
            interactions[p2] = set()
        if p2 not in interactions[p1]:
            interactions[p1].add(p2)
            interactions[p2].add(p1)
            data.append((p1, p2))

print('Total number of interactions:', len(data))


Total number of interactions: 119051


## Split training, validation and testing data


In [9]:
import numpy as np
import math

np.random.seed(seed=0) # Fix random seed for reproducibility
np.random.shuffle(data)
train_n = int(math.ceil(len(data) * 0.8))
valid_n = int(math.ceil(train_n * 0.8))
train_data = data[:valid_n]
valid_data = data[valid_n:train_n]
test_data = data[train_n:]
print('Number of training interactions:', len(train_data))
print('Number of validation interactions:', len(valid_data))
print('Number of testing interactions:', len(test_data))

Number of training interactions: 76193
Number of validation interactions: 19048
Number of testing interactions: 23810


## Save the data

In [10]:
def save(filename, data):
    with open(filename, 'w') as f:
        for p1, p2 in data:
            f.write(f'{p1}\t{p2}\n')
            f.write(f'{p2}\t{p1}\n')

save(f'data/train/{org_id}.protein.links.v11.0.txt', train_data)
save(f'data/valid/{org_id}.protein.links.v11.0.txt', valid_data)
save(f'data/test/{org_id}.protein.links.v11.0.txt', test_data)

## Preprocess GO annotations


In [11]:
mapping = {}
source = {'4932': 'SGD_ID', '9606': 'Ensembl_UniProt_AC'} # mapping source

with gzip.open(f'data/{org_id}.protein.aliases.v11.0.txt.gz', 'rt') as f:
    next(f) # Skip header
    for line in f:
        string_id, p_id, sources = line.strip().split('\t')
        if source[org_id] not in sources.split():
            continue
        if p_id not in mapping:
            mapping[p_id] = set()
        mapping[p_id].add(string_id)
print('Loaded mappings', len(mapping))

Loaded mappings 10616


In [12]:
gaf_files = {'4932': 'sgd.gaf.gz', '9606': 'goa_human.gaf.gz'}
annotations = set()
with gzip.open(f'data/{gaf_files[org_id]}', 'rt') as f:
    for line in f:
        if line.startswith('!'): # Skip header
            continue
        it = line.strip().split('\t')
        p_id = it[1]
        go_id = it[4]
        if p_id not in mapping: # Not in StringDB
            continue
        s_ids = mapping[p_id]
        for s_id in s_ids:
            annotations.add((s_id, go_id))
print('Number of annotations:', len(annotations))

# Save annotations
with open(f'data/train/{org_id}.annotation.txt', 'w') as f:
    for p_id, go_id in annotations:
        f.write(f'{p_id}\t{go_id}\n')

Number of annotations: 67573


## Generate Plain Training Data

In [13]:
tdf = open(f'data/train/{org_id}.plain.nt', 'w')
# Load GO
with open('data/go.obo') as f:
    tid = ''
    for line in f:
        line = line.strip()
        if line.startswith('id:'):
            tid = line[4:]
        if not tid.startswith('GO:'):
            continue
        if line.startswith('is_a:'):
            tid2 = line[6:].split(' ! ')[0]
            tdf.write(f'<http://{tid}> <http://is_a> <http://{tid2}> .\n')
        if line.startswith('relationship:'):
            it = line[14:].split(' ! ')[0].split()
            tdf.write(f'<http://{tid}> <http://{it[0]}> <http://{it[1]}> .\n')

# Load interactions
with open(f'data/train/{org_id}.protein.links.v11.0.txt') as f:
    for line in f:
        it = line.strip().split()
        tdf.write(f'<http://{it[0]}> <http://interacts> <http://{it[1]}> .\n')

# Load annotations
with open(f'data/train/{org_id}.annotation.txt') as f:
    for line in f:
        it = line.strip().split()
        tdf.write(f'<http://{it[0]}> <http://hasFunction> <http://{it[1]}> .\n')

tdf.close()

! wc -l 'data/train/{org_id}.plain.nt'

314503 data/train/4932.plain.nt


## Generate Classes Training Data

In [14]:

! groovy GenerateTrainingDataClasses -i 'data/train/{org_id}.protein.links.v11.0.txt' -a 'data/train/{org_id}.annotation.txt' -o 'data/train/{org_id}.classes.owl'


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
